# ASTR531 HW2

### Contents
* [9.1](#9.1)
* [11.2](#11.2)
* [12.2](#12.2)
* [13.2](#13.2)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from astropy.coordinates import Distance
from scipy.optimize import leastsq

import astropy.units as u
import astropy.constants as const
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(font_scale=1.2)

## 9.1 <a id='9.1'></a>

In [2]:
def tdyn(M, R):
    rho = 3*M/(4*np.pi*R**3)
    return ((const.G*rho)**-0.5).to(u.hour)

def tkh(M, R, L):
    return (const.G*M**2/(R*L)).to(u.year)

def tnuc(M, L):
    return ((M/u.solMass)/(L/u.solLum)).decompose() * 1e10 * u.yr

def calc_timescales(M, R, L):
    print('For a star with mass {:.1f}, radius {:.2f}, and luminosity {:.3e}:'.format(M,R,L))
    print( 'Dynamical timescale : {:.3f}'.format(tdyn(M,R)) )
    print( 'Thermal timescale   : {:.3e}'.format(tkh(M,R,L)) )
    print( 'Nuclear timescale   : {:.3e}'.format(tnuc(M,L)) )
    print('-----')

In [3]:
R_wd = 0.012 * 0.6**(-1/3)

params = [(1*u.solMass,   1*u.solRad,    1*u.solLum),
          (60*u.solMass,  15*u.solRad,   10**5.9*u.solLum),
          (15*u.solMass,  3300*u.solRad, 10**5.65*u.solLum),
          (0.6*u.solMass, R_wd*u.solRad, 1e-2*u.solLum)]

for (M, R, L) in params:
    calc_timescales(M,R,L)

For a star with mass 1.0 solMass, radius 1.00 solRad, and luminosity 1.000e+00 solLum:
Dynamical timescale : 0.906 h
Thermal timescale   : 3.140e+07 yr
Nuclear timescale   : 1.000e+10 yr
-----
For a star with mass 60.0 solMass, radius 15.00 solRad, and luminosity 7.943e+05 solLum:
Dynamical timescale : 6.792 h
Thermal timescale   : 9.487e+03 yr
Nuclear timescale   : 7.554e+05 yr
-----
For a star with mass 15.0 solMass, radius 3300.00 solRad, and luminosity 4.467e+05 solLum:
Dynamical timescale : 44324.544 h
Thermal timescale   : 4.793e+00 yr
Nuclear timescale   : 3.358e+05 yr
-----
For a star with mass 0.6 solMass, radius 0.01 solRad, and luminosity 1.000e-02 solLum:
Dynamical timescale : 0.002 h
Thermal timescale   : 7.945e+10 yr
Nuclear timescale   : 6.000e+11 yr
-----


## 11.2 <a id='11.2'></a>

In [4]:
def find_beta(B, M0=1):
    M = 18.2 * (1-B)**0.5 / (B*0.6)**2
    return M0 - M

In [5]:
B_m1 = leastsq(find_beta, 0.9, args=1)[0][0]
B_m60 = leastsq(find_beta, 0.9, args=60)[0][0]
B_m1, B_m60

/Users/luxemburg/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


(0.9996093539194961, 0.6867325537487247)

In [6]:
Prad_Pgas_m1 = (1-B_m1)/B_m1
Prad_Pgas_m60 = (1-B_m60)/B_m60
Prad_Pgas_m1, Prad_Pgas_m60

(0.00039079874450168126, 0.456170957006794)

In [7]:
def find_lum(M, B):
    Ledd = 3.8e4 * M * u.solLum
    return (1-B)*Ledd

In [8]:
L_m1 = find_lum(1, B_m1)
L_m1, np.log10(L_m1.value)

(<Quantity 14.84455106 solLum>, 1.17156706784764)

In [9]:
L_m60 = find_lum(60, B_m60)
L_m60, np.log10(L_m60.value)

(<Quantity 714249.77745291 solLum>, 5.8538501137366294)

## 12.2 <a id='12.2'></a>

In [10]:
def R_start(M):
    return 100.*M * u.solRad

def R_end(M):
    return R_start(M) / 50.

def R_ms(M):
    return M**0.7 * u.solRad

def calc_radii(M):
    print('For a star with mass {:.1f} solMass:'.format(M))
    print( 'R at start of Hayashi track : {:.1f}'.format(R_start(M)) )
    print( 'R at end of Hayashi track   : {:.1f}'.format(R_end(M)) )
    print( 'R at end of PMS contraction : {:.1f}'.format(R_ms(M)) )
    print('-----')

In [11]:
for m in [0.1, 1, 10, 100]:
    calc_radii(m)

For a star with mass 0.1 solMass:
R at start of Hayashi track : 10.0 solRad
R at end of Hayashi track   : 0.2 solRad
R at end of PMS contraction : 0.2 solRad
-----
For a star with mass 1.0 solMass:
R at start of Hayashi track : 100.0 solRad
R at end of Hayashi track   : 2.0 solRad
R at end of PMS contraction : 1.0 solRad
-----
For a star with mass 10.0 solMass:
R at start of Hayashi track : 1000.0 solRad
R at end of Hayashi track   : 20.0 solRad
R at end of PMS contraction : 5.0 solRad
-----
For a star with mass 100.0 solMass:
R at start of Hayashi track : 10000.0 solRad
R at end of Hayashi track   : 200.0 solRad
R at end of PMS contraction : 25.1 solRad
-----


In [12]:
def t_hayashi(M):
    return M**-1 * 1e6 * u.yr
    
def t_pms(M):
    return 6e7 * M**-2.5 * u.yr

def calc_pms_timescales(M):
    print('For a star with mass {:.1f} solMass:'.format(M))
    print( 'Hayashi timescale : {:.3e}'.format(t_hayashi(M)) )
    print( 'PMS timescale     : {:.3e}'.format(t_pms(M)) )
    print('-----')

In [13]:
for m in [0.1, 1, 10, 100]:
    calc_pms_timescales(m)

For a star with mass 0.1 solMass:
Hayashi timescale : 1.000e+07 yr
PMS timescale     : 1.897e+10 yr
-----
For a star with mass 1.0 solMass:
Hayashi timescale : 1.000e+06 yr
PMS timescale     : 6.000e+07 yr
-----
For a star with mass 10.0 solMass:
Hayashi timescale : 1.000e+05 yr
PMS timescale     : 1.897e+05 yr
-----
For a star with mass 100.0 solMass:
Hayashi timescale : 1.000e+04 yr
PMS timescale     : 6.000e+02 yr
-----


## 13.2 <a id='13.2'></a>

In [14]:
X0, Y0 = 0.70, 0.28
X1, Y1 = 0.49, 0.49
mu0 = (2-1.25*Y0)**-1
mu1 = (2-1.25*Y1)**-1
dL = ( ((2-Y1)**-1) * mu1**4 ) / ( ((2-Y0)**-1) * mu0**4 )
dR = ( mu1**(2/3) * (1+X1)**0.05 ) / ( mu0**(2/3) * (1+X0)**0.05 ) 
dT = ( mu1**0.83 * (1+X1)**-0.5 ) / ( mu0**0.83 * (1+X0)**-0.5 ) 
dL, dR, dT

(2.278008337654002, 1.1150752308988323, 1.2333593115878574)